---
title: Bespoke/Bayesian Models
format: html
filters:
  - shinylive
---


## Preview

Not really a preview yet.

There are *many* different statistical and machine learning models that have specific niches and applications. We chose only Generalized Linear Models as they are reasonably easy to understand and very useful, but there are many resources available for other models. What Generalized Linear Models and most other statistical models have in common, however, is that they have a set structure. If that structure fits your problem well, that's great, as it's simple to apply and simple to document/explain.

There are a couple main problems with choosing an 'off the shelf' model:

-   They often do little to help explain the underlying process.
-   They are rarely easy to improve for a special case.

The heart of why these problems exists is that most statistical models are generic tools built for common patterns that typically rely on analytical solutions.

We can, of course, develop models that operate in any way we see fit. While there isn't a single right way to develop a model for a specific data generating process, it is very common to be drawn towards Bayesian statistical paradigms when doing so. Below are some advantages of Bayesian approaches:

-   They can be tightly integrated with causal models, explaining the underlying process.
-   There are very flexible and robust methods to fit the models like Hamiltonian Monte Carlo.
-   They can integrate prior information into the parameter values:
    -   This can incorporate expert input, which is especially useful when data is sparse.
    -   It can also constrain the possible solutions, reducing the extreme flexibility that may find nonsensical solutions.

## Bespoke/Bayesian

When we talk about custom models, these are frequently associated with causal diagrams and prior information... We typically wouldn't create a custom model if we didn't have a causal diagram and some prior beliefs in mind, and custom models typically do not work well without causal diagrams and prior beliefs...

### Causal Diagrams

## All the Models

Before we get on with creating bespoke models... we should mention all the generic statistical and machine learning models that may meet your needs. If you are interested in the many statistical/machine learning tools, I suggest you refer to *An Introduction to Statistical Learning* by James, Witten, Hastie, and Tibshirani.

The various statistical/machine learning tools which include generalized linear regression, decision/regression trees, and neural nets have a distinct advantage and disadvantage: 1) So long as some basic criteria are met, you just apply the model to your data. 2) So long as some basic criteria are met, you just apply the model to your data.

That was not a typo. The advantage is both in the speed in which you can perform an analysis as well as the simplicity in communicating how you performed the analysis. I say it is a disadvantage because you are not forced to think through your problem, especially the assumptions that each tool is making. For example, in linear regression, you have a implicit prior assumption that the error term is constant and normally distributed.

In this primer we will leave all these other tools to other books, and we will instead focus on techniques that allow us to mold the model to our situation. This should only be done when you have at least *some* knowledge about the data generation process you are modeling - but it doesn't take much and who tries to model data for a situation they know nothing about?

## Revisiting the Diamond Dataset

In the diamond dataset there was something quite obvious to humans that is impossible for many statistical or machine learning tools to learn - the price premium when passing the 1.0, 1.5, and 2.0 carat thresholds.

Here's a simply X-Y plot that shows us the relationship between diamond size (carats) and price:


In [ ]:
#| echo: false
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
file_path = "../datasets/diamonds.csv"
diamonds = pd.read_csv(file_path)
diamonds = diamonds[(diamonds['cut'] == 'Premium') & (diamonds['color'] == 'E')]


# Plot the relationship between carat and price
plt.figure(figsize=(10, 6))
sns.scatterplot(data=diamonds, x='carat', y='price', alpha=0.6)
plt.title('Relationship between Carat and Price of Diamonds')
plt.xlabel('Carat')
plt.ylabel('Price')
plt.grid(True)
plt.show()

## Causal Model with 1.0, 1.5, and 2.0 Cut Points

If we reason about what is influencing the price of a diamond, once we remove the artificial value that is placed for passing the 1.0, 1.5, and 2.0 carat thresholds, the value appears to be a somewhat continuous and rational combination of the relevant characteristics, specifically color, cut, clarity, and size.

We summarize our current state of knowledge of the real data generating process in a causal diagram:


```{mermaid}
graph LR
    cut --> linear_combo
    color --> linear_combo
    clarity --> linear_combo
    size --> linear_combo
    size --> carat_threshold
    linear_combo --> price
    carat_threshold --> price
```


This shows that we have two mechanisms influencing diamond price. While the cut, color, clarity, and size can be combined in a linear fashion to influence price, the size is also an input to a carat threshold that will influence price.

## Model Flexibility

Engineers are typically familiar with solving systems of equations and the necessity that the system has the appropriate *degrees of freedom*, such that it is not overly constrained so there is no solution, and it is not underconstrained such that there are an infinite number of solutions.

Statistical models that have too much flexibility can be optimized into all sorts of nonsensical solutions. Allowing a negative intercept in a linear regression model of the diamond dataset is one example, as it indicates customers will be paid to take small diamonds. I like to visualize this as a truss with beams/members that can contort into all sorts of nonsensical shapes. What we really want are beams/members that bend in reasonable ways.

We can use our knowledge of the data generating process to create reasonable **priors** for our model. This is simply saying that we already know some reasonable range of values, or even a most likely value, for the parameters of the model. Common examples are not letting a parameter take a negative value, or that previous experience suggest the parameter is near a certain value. Setting a prior does not mean we force a parameter value on the model, it just eliminates the impossible values and nudges the model towards our prior knowledge. If the data is different...

### Test

```{shinylive-python}
#| standalone: true
#| viewerHeight: 700

from shiny import App, ui, render, reactive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import statsmodels.api as sm
import patsy

plt.switch_backend('agg')

# --------------------------------------------------------------------
# 1) Load/prepare data
# --------------------------------------------------------------------
data_str = """
ID,carat,cut,color,clarity,depth,table,price,x,y,z
51657,0.3,Ideal,G,VS2,62.3,58.0,545,4.26,4.28,2.66
34838,0.3,Premium,G,VVS2,60.8,58.0,878,4.38,4.34,2.65
9718,0.3,Ideal,H,VVS2,62.1,54.0,590,4.32,4.35,2.69
46635,0.3,Very Good,E,SI1,62.7,60.0,526,4.24,4.28,2.67
31852,0.3,Premium,G,VS1,62.2,59.0,776,4.28,4.24,2.65
40942,0.27,Ideal,H,VS1,62.3,54.0,500,4.16,4.19,2.6
49960,0.3,Good,H,SI1,63.7,56.0,540,4.22,4.2,2.68
30300,0.3,Very Good,D,SI2,61.0,61.0,447,4.25,4.31,2.61
15051,0.3,Ideal,F,VS2,61.4,57.0,605,4.34,4.36,2.67
32272,0.3,Very Good,G,VVS1,62.9,57.0,789,4.26,4.3,2.69
16695,0.3,Very Good,H,SI1,62.6,58.0,421,4.22,4.28,2.66
32358,0.3,Good,G,VVS1,63.1,56.0,789,4.25,4.28,2.69
3393,0.27,Very Good,E,VVS2,59.4,64.0,567,4.16,4.19,2.48
16027,0.3,Premium,I,VS1,60.5,60.0,608,4.33,4.3,2.61
5721,0.25,Very Good,E,VVS2,60.9,59.0,575,4.03,4.11,2.48
34695,0.3,Ideal,F,IF,61.7,56.0,873,4.31,4.35,2.67
28794,0.27,Very Good,F,VVS2,61.3,57.0,682,4.14,4.18,2.54
32496,0.3,Good,F,IF,58.8,61.0,796,4.35,4.39,2.57
16359,0.3,Good,D,VS2,64.1,57.0,608,4.25,4.21,2.71
31973,0.3,Very Good,I,VS2,60.5,55.0,453,4.34,4.37,2.63
51312,0.31,Ideal,G,VS2,59.1,57.0,544,4.45,4.48,2.64
27844,0.31,Very Good,G,VS2,63.2,58.0,651,4.3,4.28,2.71
37309,0.31,Ideal,F,IF,62.2,56.0,979,4.31,4.34,2.69
16685,0.31,Ideal,H,SI2,61.1,56.0,421,4.4,4.42,2.69
35803,0.31,Premium,F,IF,61.9,58.0,914,4.36,4.39,2.71
30256,0.31,Very Good,E,VVS1,60.4,61.0,725,4.34,4.4,2.64
36008,0.31,Ideal,F,IF,61.2,56.0,921,4.37,4.42,2.69
30803,0.31,Good,F,VVS1,63.6,61.0,742,4.21,4.25,2.69
32676,0.31,Premium,G,VS1,62.4,59.0,802,4.34,4.32,2.7
35593,0.31,Ideal,H,VVS1,62.2,54.0,907,4.39,4.36,2.72
20386,0.31,Premium,G,VS1,59.5,59.0,625,4.4,4.47,2.64
34570,0.31,Ideal,G,IF,61.0,55.0,871,4.39,4.42,2.69
33609,0.31,Ideal,D,SI2,62.0,56.0,462,4.33,4.35,2.69
32609,0.31,Premium,H,VVS2,61.4,59.0,802,4.38,4.35,2.68
32723,0.31,Ideal,F,VS2,62.7,57.0,802,4.34,4.3,2.71
44998,0.31,Premium,I,SI1,62.3,59.0,523,4.32,4.29,2.68
38803,0.31,Very Good,G,VVS1,63.1,56.0,1046,4.35,4.33,2.74
43285,0.31,Very Good,D,SI1,60.4,60.0,507,4.4,4.44,2.67
33131,0.31,Very Good,E,VVS2,60.8,55.0,816,4.38,4.43,2.68
35157,0.31,Very Good,G,IF,61.6,54.0,891,4.4,4.43,2.72
37580,0.32,Premium,D,VVS2,61.5,60.0,990,4.41,4.37,2.7
33506,0.32,Premium,G,VS1,62.5,60.0,828,4.35,4.29,2.7
26341,0.32,Ideal,H,VVS2,61.7,56.0,645,4.37,4.42,2.71
33033,0.32,Ideal,G,VVS1,61.4,57.0,814,4.39,4.41,2.7
36290,0.32,Ideal,G,SI1,61.3,57.0,477,4.37,4.4,2.69
36284,0.32,Ideal,D,SI2,62.4,54.0,477,4.38,4.4,2.74
13404,0.32,Very Good,F,VS2,61.2,58.0,602,4.38,4.41,2.69
30954,0.32,Ideal,I,VS2,62.5,55.0,449,4.38,4.39,2.74
29634,0.32,Ideal,J,VS1,62.0,54.7,442,4.39,4.42,2.73
30129,0.32,Ideal,G,VS2,61.8,57.0,720,4.4,4.37,2.71
46963,0.32,Good,F,SI1,61.6,60.1,528,4.38,4.4,2.71
32783,0.32,Ideal,D,VVS2,61.2,56.0,803,4.39,4.43,2.7
20012,0.32,Good,G,SI2,63.4,55.0,421,4.32,4.35,2.75
34133,0.32,Ideal,F,VVS1,60.4,57.0,854,4.41,4.43,2.67
27865,0.32,Ideal,G,SI1,61.4,56.0,653,4.44,4.42,2.72
29989,0.32,Ideal,F,VS1,61.0,54.0,716,4.42,4.44,2.7
30145,0.32,Premium,G,VS2,62.8,58.0,720,4.35,4.31,2.72
31320,0.32,Ideal,D,VS2,62.6,55.0,758,4.37,4.39,2.74
35896,0.32,Ideal,G,IF,61.7,54.0,918,4.42,4.46,2.74
50304,0.32,Very Good,G,VS2,62.3,55.0,544,4.38,4.41,2.73
32501,0.33,Premium,G,VS1,61.6,57.0,797,4.51,4.42,2.75
29919,0.33,Ideal,H,VVS1,61.8,55.0,713,4.42,4.44,2.74
37434,0.33,Good,G,IF,57.9,60.0,984,4.55,4.57,2.64
42419,0.33,Ideal,E,VVS1,61.9,57.0,1312,4.43,4.46,2.75
30338,0.34,Premium,F,SI1,59.4,62.0,727,4.59,4.54,2.71
23380,0.33,Very Good,G,SI1,63.2,57.0,631,4.44,4.39,2.79
18704,0.35,Very Good,I,VVS2,61.3,56.0,620,4.52,4.54,2.78
31350,0.34,Ideal,E,VS2,61.8,54.0,760,4.49,4.5,2.78
34543,0.35,Ideal,H,IF,61.5,57.0,868,4.55,4.58,2.8
13389,0.35,Premium,D,SI1,61.5,58.0,601,4.53,4.55,2.79
36970,0.34,Ideal,D,VS1,60.7,57.0,961,4.55,4.51,2.75
37025,0.33,Ideal,G,VVS2,62.5,54.0,965,4.45,4.41,2.77
30831,0.33,Premium,I,VVS2,61.5,58.0,743,4.45,4.43,2.73
36287,0.34,Very Good,E,SI2,61.7,61.0,477,4.47,4.51,2.77
34161,0.33,Premium,G,VS1,60.5,58.0,854,4.49,4.43,2.7
30719,0.35,Fair,E,VVS2,66.2,61.0,738,4.4,4.36,2.9
33204,0.35,Ideal,G,VVS2,61.8,55.0,820,4.53,4.56,2.81
26014,0.35,Premium,D,SI1,60.9,58.0,644,4.52,4.55,2.76
27052,0.33,Ideal,I,VVS1,62.2,54.0,646,4.43,4.45,2.76
34181,0.33,Ideal,G,VS1,62.1,56.0,854,4.42,4.4,2.74
28218,0.4,Premium,D,SI2,62.1,60.0,666,4.69,4.75,2.93
39564,0.4,Premium,G,VS1,62.2,55.0,1080,4.83,4.69,2.96
33662,0.36,Ideal,E,VS1,61.4,54.0,835,4.59,4.63,2.83
36552,0.4,Ideal,E,SI1,60.5,57.0,945,4.81,4.77,2.9
37369,0.4,Very Good,F,VS1,60.4,61.0,982,4.74,4.77,2.87
41873,0.38,Ideal,D,VVS2,61.5,56.0,1257,4.66,4.64,2.86
35767,0.4,Premium,E,VS2,60.7,60.0,912,4.7,4.75,2.87
27792,0.37,Premium,G,VS2,61.3,60.0,649,4.6,4.63,2.83
41652,0.4,Ideal,E,VVS2,62.1,56.0,1238,4.73,4.7,2.93
37757,0.38,Premium,D,VS2,61.6,59.0,998,4.66,4.62,2.86
36266,0.37,Ideal,H,IF,61.7,53.0,936,4.66,4.68,2.88
37328,0.4,Premium,G,VVS2,61.3,59.0,980,4.78,4.74,2.92
38250,0.36,Ideal,D,VS1,62.8,55.0,1018,4.55,4.52,2.85
35397,0.38,Good,F,VS2,62.4,54.3,899,4.6,4.65,2.89
31021,0.37,Premium,I,VS1,61.4,59.0,749,4.61,4.55,2.81
30667,0.4,Very Good,I,VS1,63.0,56.0,737,4.68,4.72,2.96
39618,0.37,Very Good,H,SI1,62.6,63.0,491,4.6,4.5,2.85
30669,0.4,Premium,F,SI1,62.5,59.0,737,4.67,4.71,2.93
17728,0.39,Ideal,E,SI2,61.0,55.0,614,4.74,4.77,2.9
35328,0.38,Ideal,H,VVS2,62.1,54.0,898,4.62,4.66,2.88
33367,0.41,Ideal,G,VS2,61.4,55.0,827,4.75,4.8,2.93
39486,0.41,Ideal,E,VS1,62.1,55.0,1079,4.75,4.78,2.96
31789,0.42,Ideal,E,SI1,61.3,57.0,773,4.79,4.81,2.94
33930,0.41,Good,G,VVS1,63.6,56.0,844,4.72,4.74,3.01
41724,0.41,Ideal,H,IF,61.8,55.0,1243,4.79,4.76,2.95
42168,0.41,Premium,D,VS1,59.3,58.0,1286,4.87,4.85,2.88
30052,0.41,Premium,G,SI1,59.1,58.0,719,4.83,4.88,2.87
41467,0.41,Premium,G,VVS1,61.0,61.0,1230,4.75,4.72,2.89
35509,0.41,Premium,E,SI1,62.8,58.0,904,4.77,4.72,2.98
24390,0.41,Very Good,E,SI2,63.0,57.0,638,4.7,4.73,2.97
35351,0.42,Ideal,H,SI1,62.4,57.0,898,4.79,4.76,2.98
37077,0.41,Premium,F,SI1,62.6,55.0,969,4.78,4.74,2.98
36978,0.42,Premium,G,VVS2,61.6,60.0,963,4.8,4.85,2.97
28454,0.41,Ideal,G,SI1,62.2,56.0,671,4.75,4.77,2.96
43252,0.42,Premium,G,IF,60.2,59.0,1400,4.8,4.87,2.91
41015,0.41,Very Good,F,VVS1,62.7,59.0,1186,4.75,4.78,2.99
37665,0.42,Premium,E,SI1,61.6,59.0,992,4.85,4.83,2.98
40213,0.41,Ideal,D,SI1,61.8,56.0,1122,4.78,4.73,2.94
37909,0.41,Ideal,F,VS1,60.8,56.0,1007,4.76,4.79,2.92
39436,0.41,Ideal,D,VS2,62.2,54.0,1076,4.81,4.77,2.98
46182,0.5,Ideal,I,VVS1,61.6,56.0,1747,5.1,5.13,3.15
38815,0.45,Premium,F,SI1,61.1,58.0,1046,4.97,4.95,3.03
41423,0.46,Ideal,H,VVS1,62.3,54.0,1227,4.96,4.99,3.1
50341,0.5,Ideal,D,VS2,61.1,57.0,2243,5.11,5.13,3.13
43455,0.5,Premium,G,VS2,61.5,57.0,1415,5.12,5.09,3.14
35239,0.43,Very Good,E,SI1,63.4,56.0,894,4.82,4.8,3.05
41838,0.44,Ideal,F,VVS2,60.9,55.0,1253,4.96,4.92,3.01
37303,0.5,Premium,G,SI2,60.7,57.0,978,5.15,5.07,3.1
37391,0.5,Ideal,I,SI1,62.0,55.0,982,5.08,5.11,3.16
38196,0.5,Very Good,D,SI2,63.1,56.0,1015,5.05,4.96,3.16
33009,0.43,Premium,F,SI2,58.3,62.0,813,4.97,4.91,2.88
43403,0.46,Ideal,G,VVS1,62.0,54.0,1412,4.97,5.0,3.09
44797,0.5,Very Good,E,VS2,61.5,56.0,1624,5.07,5.11,3.13
32446,0.43,Very Good,H,VS2,61.9,55.0,792,4.8,4.95,3.02
39507,0.5,Ideal,F,SI2,61.7,55.0,1080,5.13,5.15,3.17
42348,0.46,Ideal,H,SI1,61.2,56.0,1299,4.97,5.0,3.05
49157,0.5,Very Good,G,VVS1,63.3,56.0,2070,5.1,5.07,3.22
39697,0.48,Good,G,VS2,65.4,59.0,1088,4.79,4.88,3.16
47045,0.5,Premium,D,VS2,59.7,57.0,1819,5.13,5.08,3.05
38353,0.47,Very Good,F,SI1,61.1,61.0,1021,4.97,5.01,3.05
49694,0.51,Very Good,E,VVS2,62.8,57.0,2146,5.06,5.1,3.19
39316,0.53,Very Good,G,SI2,60.8,58.0,1070,5.19,5.21,3.16
44608,0.53,Premium,E,SI1,61.9,56.0,1607,5.22,5.19,3.22
47613,0.53,Ideal,G,VVS2,60.4,55.0,1881,5.26,5.3,3.19
44575,0.53,Ideal,E,VS2,62.5,57.0,1607,5.16,5.18,3.23
49934,0.51,Premium,E,VVS2,62.1,57.0,2185,5.18,5.15,3.21
41199,0.51,Very Good,D,SI2,60.3,57.0,1204,5.15,5.17,3.11
47601,0.52,Ideal,G,VVS2,60.8,57.0,1878,5.2,5.17,3.15
48545,0.52,Ideal,I,IF,60.2,56.0,1988,5.23,5.27,3.16
41422,0.52,Very Good,F,SI1,62.3,55.0,1227,5.14,5.17,3.21
48904,0.51,Very Good,F,VVS2,62.0,56.0,2041,5.1,5.15,3.17
43201,0.53,Good,G,VS2,63.4,58.0,1395,5.13,5.16,3.26
46534,0.51,Ideal,G,VS1,62.5,57.0,1781,5.14,5.07,3.19
43116,0.52,Very Good,H,VS2,63.5,58.0,1385,5.12,5.11,3.25
36885,0.51,Good,I,SI1,63.1,56.0,959,5.06,5.14,3.22
44284,0.51,Ideal,G,VS1,62.5,57.0,1577,5.08,5.1,3.18
37127,0.52,Ideal,D,I1,61.1,57.0,971,5.18,5.2,3.17
48116,0.52,Ideal,G,VVS1,61.9,54.4,1936,5.15,5.18,3.2
44258,0.51,Ideal,H,VVS2,61.0,57.0,1574,5.22,5.18,3.17
46475,0.51,Ideal,H,VVS1,61.4,55.0,1776,5.13,5.16,3.16
46460,0.54,Ideal,F,VS1,61.1,57.0,1774,5.28,5.3,3.23
50067,0.54,Ideal,F,VS1,61.5,55.0,2202,5.26,5.27,3.24
43563,0.58,Fair,G,VS2,65.0,56.0,1430,5.23,5.17,3.38
47010,0.56,Ideal,E,VS2,60.9,56.0,1819,5.32,5.35,3.25
41886,0.54,Ideal,I,VS2,61.1,55.0,1259,5.27,5.31,3.23
42007,0.59,Ideal,F,SI2,61.8,55.0,1265,5.41,5.44,3.35
48843,0.55,Ideal,E,VS2,62.5,56.0,2030,5.26,5.23,3.28
52201,0.54,Ideal,E,VVS2,61.9,54.5,2479,5.22,5.25,3.23
49498,0.56,Ideal,H,VVS2,61.8,56.0,2118,5.28,5.33,3.28
52348,0.55,Ideal,E,VVS2,61.4,56.0,2499,5.28,5.31,3.25
50508,0.54,Ideal,G,IF,62.3,56.0,2271,5.19,5.21,3.24
46004,0.54,Ideal,D,VS2,61.2,56.0,1725,5.24,5.28,3.22
46440,0.54,Ideal,F,VS1,60.9,57.0,1772,5.21,5.26,3.19
45822,0.56,Good,F,VS1,63.2,61.0,1712,5.2,5.28,3.3
46373,0.58,Ideal,G,VS2,61.9,55.0,1761,5.33,5.36,3.31
41799,0.6,Very Good,E,SI2,63.2,60.0,1250,5.32,5.28,3.35
45126,0.59,Very Good,E,SI1,62.9,58.0,1652,5.31,5.34,3.35
43185,0.54,Very Good,G,SI1,63.2,58.0,1392,5.15,5.16,3.26
45719,0.56,Ideal,E,SI1,62.7,57.0,1698,5.27,5.23,3.29
42200,0.56,Premium,G,SI1,61.1,61.0,1287,5.31,5.29,3.24
3262,0.7,Ideal,F,VS1,60.3,57.0,3359,5.74,5.79,3.47
51331,0.7,Very Good,F,VS2,62.3,56.0,2362,5.66,5.71,3.54
50892,0.7,Premium,G,VS2,60.8,58.0,2317,5.75,5.8,3.51
46073,0.63,Premium,F,SI1,59.1,57.0,1736,5.64,5.6,3.32
53792,0.7,Very Good,E,SI1,62.1,60.0,2730,5.62,5.66,3.5
1543,0.7,Very Good,D,VS1,63.4,59.0,3001,5.58,5.55,3.53
2516,0.7,Ideal,E,VS2,60.5,59.0,3201,5.72,5.75,3.47
52766,0.7,Very Good,G,VS2,58.7,53.0,2563,5.83,5.86,3.43
52504,0.7,Good,D,SI1,58.0,60.0,2525,5.79,5.93,3.4
52161,0.7,Premium,D,SI1,60.8,58.0,2473,5.79,5.66,3.48
44158,0.7,Fair,F,SI2,66.4,56.0,1564,5.51,5.42,3.63
46845,0.64,Premium,E,SI1,61.3,58.0,1811,5.57,5.53,3.4
47260,0.7,Premium,J,VS2,61.2,60.0,1843,5.7,5.73,3.5
2424,0.63,Ideal,E,VVS1,61.1,58.0,3181,5.49,5.54,3.37
48887,0.7,Very Good,F,SI2,59.6,61.0,2039,5.8,5.88,3.48
51599,0.7,Good,I,VVS2,63.3,55.0,2394,5.61,5.67,3.57
46198,0.7,Fair,I,SI1,65.2,58.0,1749,5.6,5.56,3.64
49877,0.7,Premium,H,SI1,60.9,62.0,2176,5.72,5.67,3.47
52012,0.7,Good,D,SI1,59.9,63.0,2444,5.74,5.81,3.46
2986,0.7,Ideal,G,VS1,60.8,56.0,3300,5.73,5.8,3.51
277,0.71,Very Good,E,VS2,60.7,56.0,2795,5.81,5.82,3.53
809,0.71,Premium,D,SI1,59.7,59.0,2863,5.82,5.8,3.47
52887,0.72,Premium,H,VS2,60.7,59.0,2583,5.84,5.8,3.53
946,0.72,Very Good,G,VVS2,62.5,58.0,2889,5.68,5.72,3.56
51695,0.71,Very Good,I,VVS2,59.5,60.0,2400,5.82,5.87,3.48
48158,0.72,Very Good,H,SI2,63.5,58.0,1942,5.65,5.68,3.6
51672,0.72,Ideal,E,SI2,61.9,55.0,2398,5.76,5.78,3.57
3806,0.72,Ideal,E,VS1,62.5,57.0,3465,5.73,5.76,3.59
51150,0.71,Premium,F,SI2,62.0,59.0,2343,5.68,5.65,3.51
694,0.71,Premium,F,VS2,62.6,58.0,2853,5.67,5.7,3.56
50848,0.72,Premium,H,SI1,62.2,57.0,2311,5.75,5.72,3.57
45878,0.71,Premium,G,SI2,59.9,59.0,1717,5.79,5.82,3.48
49717,0.72,Premium,I,SI1,61.5,59.0,2148,5.73,5.78,3.54
2140,0.72,Ideal,H,VVS1,61.4,56.0,3124,5.79,5.77,3.55
1181,0.71,Ideal,G,VS1,62.7,57.0,2930,5.69,5.73,3.58
50722,0.71,Premium,I,VS2,62.1,59.0,2294,5.7,5.73,3.55
53191,0.71,Premium,F,SI1,62.7,57.0,2633,5.68,5.65,3.55
48876,0.71,Very Good,F,SI2,63.3,56.0,2036,5.68,5.73,3.61
3635,0.71,Ideal,G,VS1,60.7,57.0,3431,5.76,5.8,3.51
51843,0.71,Very Good,E,SI2,62.2,58.0,2423,5.65,5.7,3.53
53670,0.74,Very Good,H,VS1,61.9,59.1,2709,5.74,5.77,3.56
7260,0.9,Ideal,F,SI2,61.5,56.0,4198,6.24,6.18,3.82
7909,0.9,Ideal,G,SI2,60.7,57.0,4314,6.19,6.33,3.8
8568,0.9,Premium,F,SI1,61.4,55.0,4435,6.18,6.16,3.79
1110,0.8,Very Good,F,SI1,63.5,55.0,2914,5.86,5.89,3.73
53096,0.75,Ideal,I,VS1,63.0,57.0,2613,5.8,5.82,3.66
1207,0.76,Premium,E,SI1,58.3,62.0,2937,6.12,5.95,3.52
580,0.78,Ideal,I,VS2,61.8,55.0,2834,5.92,5.95,3.67
47891,0.74,Very Good,J,SI1,62.2,59.0,1913,5.74,5.81,3.59
1486,0.77,Premium,E,SI1,61.7,58.0,2988,5.86,5.9,3.63
53472,0.76,Ideal,E,SI2,61.5,55.0,2680,5.88,5.93,3.63
4245,0.84,Good,E,SI1,61.9,61.0,3577,6.03,6.05,3.74
4671,0.76,Ideal,G,VVS1,62.0,54.7,3671,5.83,5.87,3.62
1813,0.78,Very Good,E,SI1,60.9,57.0,3055,5.93,5.97,3.62
682,0.75,Ideal,J,SI1,61.5,56.0,2850,5.83,5.87,3.6
113,0.9,Premium,I,VS2,63.0,58.0,2761,6.16,6.12,3.87
3221,0.9,Very Good,G,SI2,63.5,57.0,3350,6.09,6.13,3.88
9439,0.9,Very Good,H,VVS2,63.7,57.0,4592,6.09,6.02,3.86
53398,0.83,Ideal,H,SI2,61.1,59.0,2666,6.05,6.1,3.71
4108,0.74,Ideal,G,VVS1,62.1,54.0,3537,5.8,5.83,3.61
4215,0.91,Very Good,H,VS2,63.1,56.0,3567,6.2,6.13,3.89
9572,1.0,Premium,D,SI2,62.2,61.0,4626,6.36,6.3,3.94
8097,0.95,Premium,D,SI2,60.1,61.0,4341,6.37,6.35,3.82
14644,1.0,Premium,H,VVS2,61.4,59.0,5914,6.49,6.45,3.97
12007,1.0,Good,G,VS2,63.8,59.0,5148,6.26,6.34,4.02
3802,1.0,Very Good,J,SI1,61.9,62.0,3465,6.33,6.36,3.93
6503,0.97,Fair,F,SI1,56.4,66.0,4063,6.59,6.54,3.7
9575,1.0,Premium,D,SI2,59.4,60.0,4626,6.56,6.48,3.87
4748,0.92,Premium,F,SI1,62.6,59.0,3684,6.23,6.19,3.89
10565,1.0,Premium,G,SI1,60.8,58.0,4816,6.48,6.45,3.93
9806,0.91,Very Good,E,SI2,63.2,56.0,4668,6.08,6.14,3.86
13270,1.0,Good,G,VS2,56.6,61.0,5484,6.65,6.61,3.75
18435,1.0,Good,D,VS1,57.8,61.0,7500,6.62,6.56,3.81
3591,0.91,Premium,G,SI2,61.3,60.0,3423,6.17,6.2,3.79
5447,1.0,Fair,H,SI1,55.2,64.0,3830,6.69,6.64,3.68
15947,1.0,Premium,G,VS1,62.4,60.0,6377,6.39,6.37,3.98
10800,1.0,Good,H,VS2,63.7,59.0,4861,6.3,6.26,4.0
5849,1.0,Premium,H,SI2,61.3,58.0,3920,6.45,6.41,3.94
8315,0.91,Very Good,D,SI1,63.5,56.0,4389,6.13,6.18,3.91
4151,0.91,Premium,F,SI2,61.0,51.0,3546,6.24,6.21,3.8
9426,1.01,Very Good,D,SI2,62.8,59.0,4588,6.34,6.44,4.01
10581,1.01,Very Good,D,SI1,59.1,61.0,4821,6.46,6.5,3.83
15174,1.01,Very Good,H,VVS2,63.3,57.0,6097,6.39,6.35,4.03
5937,1.01,Very Good,F,SI2,60.8,63.0,3945,6.32,6.38,3.86
9236,1.01,Good,H,SI1,63.3,58.0,4559,6.37,6.4,4.04
15117,1.01,Premium,D,SI1,61.8,58.0,6075,6.42,6.37,3.95
7700,1.01,Fair,F,SI1,67.2,60.0,4276,6.06,6.0,4.05
9013,1.01,Premium,H,SI1,61.3,58.0,4513,6.47,6.39,3.94
15740,1.01,Ideal,G,VS2,60.6,58.0,6295,6.44,6.5,3.92
11337,1.01,Good,F,SI1,63.7,57.0,4989,6.4,6.35,4.06
15199,1.01,Very Good,G,VS2,61.9,56.0,6105,6.34,6.42,3.95
10942,1.01,Very Good,F,SI1,59.7,61.0,4899,6.49,6.55,3.89
4744,1.01,Very Good,G,SI2,62.0,58.0,3682,6.41,6.46,3.99
18733,1.01,Very Good,D,VS2,62.7,57.0,7652,6.36,6.39,4.0
15525,1.01,Very Good,E,VS2,63.0,60.0,6221,6.32,6.35,3.99
16288,1.01,Very Good,E,VS2,63.3,60.0,6516,6.33,6.3,4.0
11015,1.01,Very Good,G,SI1,60.6,57.0,4916,6.49,6.52,3.94
16798,1.01,Premium,E,VS2,60.4,57.0,6697,6.49,6.45,3.91
11293,1.01,Ideal,H,SI1,62.3,55.0,4977,6.43,6.37,3.99
13505,1.01,Ideal,D,SI1,61.2,57.0,5543,6.47,6.44,3.95
13562,1.02,Very Good,E,SI1,59.2,56.0,5553,6.57,6.63,3.91
9083,1.03,Premium,E,SI2,61.0,60.0,4522,6.53,6.46,3.96
9159,1.02,Very Good,E,SI2,63.3,58.0,4540,6.31,6.4,4.02
10316,1.03,Very Good,G,SI1,63.2,58.0,4764,6.43,6.38,4.05
12600,1.02,Very Good,F,SI1,60.9,57.0,5287,6.52,6.56,3.98
15398,1.02,Very Good,G,VS2,63.4,59.0,6169,6.32,6.3,4.0
8405,1.03,Ideal,I,SI1,63.3,57.0,4401,6.37,6.46,4.06
17889,1.04,Ideal,D,VS2,61.9,55.0,7220,6.5,6.52,4.03
7153,1.04,Very Good,F,SI2,62.3,58.0,4181,6.44,6.5,4.03
16983,1.03,Premium,F,VS1,61.7,56.0,6783,6.49,6.47,4.0
11198,1.02,Premium,H,VS2,60.0,58.0,4958,6.56,6.5,3.92
5865,1.03,Ideal,J,SI1,62.6,57.0,3922,6.45,6.43,4.03
15016,1.02,Very Good,D,SI1,62.8,56.0,6047,6.39,6.44,4.03
7502,1.04,Premium,E,SI2,61.6,59.0,4240,6.57,6.55,4.04
14328,1.03,Ideal,D,SI1,61.2,55.0,5804,6.51,6.57,4.0
8632,1.02,Premium,G,SI1,62.6,59.0,4449,6.43,6.38,4.01
7041,1.02,Ideal,F,SI2,62.1,56.0,4162,6.41,6.44,3.99
21809,1.03,Ideal,F,VVS1,61.3,54.0,9881,6.56,6.62,4.04
48885,1.04,Fair,I,I1,67.3,56.0,2037,6.34,6.23,4.22
16635,1.02,Premium,F,VS2,62.4,59.0,6652,6.4,6.45,4.01
15538,1.09,Ideal,I,VS1,61.8,55.0,6225,6.59,6.62,4.08
18682,1.11,Ideal,G,VS1,61.5,58.0,7639,6.7,6.66,4.11
7580,1.06,Very Good,I,SI1,62.8,56.0,4255,6.47,6.52,4.08
8646,1.06,Premium,F,SI2,62.4,58.0,4452,6.54,6.5,4.07
20512,1.11,Ideal,G,VVS2,63.1,57.0,8843,6.55,6.6,4.15
13460,1.13,Very Good,G,SI1,63.1,58.0,5526,6.65,6.59,4.18
11822,1.07,Ideal,I,SI1,61.7,56.0,5093,6.59,6.57,4.06
19907,1.09,Premium,G,VVS2,59.5,61.0,8454,6.74,6.7,4.0
16948,1.08,Ideal,G,VS2,60.3,59.0,6769,6.62,6.64,4.0
15439,1.05,Premium,G,VS2,61.8,58.0,6181,6.59,6.52,4.05
17304,1.09,Ideal,G,VS1,62.4,57.0,6934,6.55,6.63,4.11
14807,1.11,Ideal,E,SI2,60.6,56.0,5962,6.76,6.78,4.1
21425,1.07,Ideal,G,IF,61.5,57.0,9532,6.59,6.54,4.04
4661,1.13,Ideal,H,I1,61.1,56.0,3669,6.77,6.71,4.12
16344,1.1,Ideal,G,VS1,61.3,54.0,6535,6.69,6.65,4.09
11847,1.05,Ideal,I,VS1,61.5,55.0,5101,6.56,6.61,4.05
16867,1.07,Premium,G,VS1,62.0,58.0,6730,6.59,6.53,4.07
21535,1.12,Ideal,F,VVS2,61.4,57.0,9634,6.69,6.66,4.1
8220,1.09,Very Good,J,VS2,62.3,59.0,4372,6.56,6.63,4.11
18833,1.12,Ideal,G,VS1,61.6,55.0,7716,6.69,6.72,4.13
13956,1.16,Very Good,G,SI1,60.7,59.0,5678,6.74,6.87,4.13
20531,1.23,Premium,F,VS2,59.6,58.0,8855,6.94,7.02,4.16
12498,1.15,Very Good,E,SI2,60.0,59.0,5257,6.78,6.82,4.08
14003,1.2,Premium,I,VS2,62.6,58.0,5699,6.77,6.72,4.22
22973,1.2,Premium,F,VVS2,62.2,58.0,11021,6.83,6.78,4.23
8795,1.21,Premium,F,SI2,61.8,59.0,4472,6.82,6.77,4.2
18812,1.24,Ideal,H,VS2,60.1,59.0,7701,6.99,7.03,4.21
26565,1.2,Ideal,E,VVS1,61.8,56.0,16256,6.78,6.87,4.22
20122,1.24,Ideal,G,VS1,61.9,54.0,8584,6.89,6.92,4.27
12313,1.24,Ideal,I,SI2,61.9,57.0,5221,6.87,6.92,4.27
15155,1.21,Premium,F,SI2,59.0,60.0,6092,6.99,6.94,4.11
18869,1.22,Ideal,H,VS1,60.4,57.0,7738,6.86,6.89,4.15
16067,1.2,Premium,H,VS2,62.5,58.0,6416,6.77,6.73,4.23
10468,1.21,Very Good,I,SI2,62.1,59.0,4791,6.8,6.86,4.24
12328,1.2,Very Good,J,VS1,62.9,60.0,5226,6.64,6.69,4.19
7885,1.21,Premium,F,SI2,62.4,60.0,4310,6.77,6.73,4.21
23561,1.21,Ideal,G,VVS1,61.5,56.0,11572,6.83,6.89,4.22
20700,1.22,Very Good,G,VVS2,61.9,58.0,8975,6.84,6.85,4.24
20006,1.2,Ideal,G,VS1,62.4,57.0,8545,6.78,6.8,4.24
15584,1.2,Premium,F,SI1,62.4,58.0,6250,6.81,6.75,4.23
24545,1.51,Premium,G,VS1,62.4,60.0,12831,7.3,7.34,4.57
26041,1.5,Premium,D,VS2,61.8,60.0,15240,7.37,7.3,4.53
25000,1.5,Very Good,G,VS2,61.1,60.0,13528,7.4,7.3,4.49
6157,1.25,Fair,H,SI2,64.4,58.0,3990,6.82,6.71,4.36
10957,1.25,Ideal,H,SI2,61.6,54.0,4900,6.94,6.88,4.25
14113,1.4,Premium,G,SI2,60.6,58.0,5723,7.26,7.22,4.39
15653,1.26,Ideal,F,SI2,62.7,58.0,6277,6.91,6.87,4.32
12682,1.26,Ideal,J,VS2,63.2,57.0,5306,6.86,6.81,4.32
21426,1.5,Very Good,I,VS2,63.3,55.0,9533,7.3,7.26,4.61
22405,1.5,Good,G,SI1,64.2,58.0,10428,7.14,7.2,4.6
20409,1.5,Premium,F,SI1,62.1,60.0,8770,7.32,7.27,4.53
19944,1.5,Premium,H,SI2,62.3,60.0,8490,7.22,7.3,4.52
16950,1.5,Very Good,H,SI2,63.3,57.0,6770,7.27,7.21,4.59
19527,1.5,Good,I,SI1,62.9,60.0,8161,7.12,7.16,4.49
19250,1.33,Premium,H,VS2,60.7,59.0,7982,7.08,7.13,4.31
15127,1.32,Very Good,J,VS2,62.1,57.0,6079,7.01,7.04,4.36
24098,1.5,Very Good,E,SI1,59.3,60.0,12247,7.4,7.5,4.42
16218,1.33,Very Good,H,SI2,62.5,58.0,6482,7.04,6.97,4.38
20898,1.51,Premium,I,VS2,63.0,60.0,9116,7.3,7.25,4.58
21870,1.25,Ideal,D,VS2,62.6,56.0,9933,6.84,6.87,4.29
25222,1.7,Ideal,H,VS1,62.4,55.0,13823,7.61,7.69,4.77
24230,1.62,Good,H,VS2,61.5,60.8,12429,7.48,7.53,4.62
22614,1.52,Good,F,SI1,63.6,54.0,10664,7.33,7.22,4.63
22933,1.52,Ideal,I,VVS1,61.9,56.0,10968,7.34,7.37,4.55
19386,1.55,Ideal,I,SI2,60.7,60.0,8056,7.49,7.46,4.54
20220,1.54,Premium,J,VVS2,61.1,59.0,8652,7.45,7.4,4.54
24512,1.53,Ideal,E,SI1,62.3,54.2,12791,7.35,7.38,4.59
21122,1.54,Very Good,J,VS1,63.5,57.0,9285,7.27,7.37,4.65
23411,1.67,Premium,I,VS1,61.1,58.0,11400,7.69,7.6,4.67
19348,1.56,Good,I,SI2,58.5,61.0,8048,7.58,7.63,4.45
19758,1.56,Premium,J,VS1,61.1,59.0,8324,7.49,7.52,4.58
25204,1.52,Very Good,D,VS2,62.4,58.0,13799,7.23,7.28,4.53
27338,1.7,Ideal,F,VS2,62.3,56.0,17892,7.61,7.65,4.75
27530,1.7,Ideal,G,VVS1,61.0,56.0,18279,7.62,7.67,4.66
25164,1.7,Premium,F,VS2,62.5,61.0,13737,7.54,7.45,4.69
24018,1.7,Ideal,D,SI1,60.0,54.0,12190,7.76,7.71,4.64
15979,1.7,Ideal,H,I1,61.3,55.0,6397,7.7,7.63,4.7
25184,1.52,Ideal,G,VS2,62.1,56.0,13768,7.39,7.34,4.57
20248,1.55,Ideal,H,SI2,62.1,57.0,8678,7.39,7.43,4.6
17928,1.53,Ideal,G,SI2,61.7,57.0,7240,7.44,7.41,4.58
24211,2.14,Ideal,H,SI2,61.9,57.0,12400,8.34,8.28,5.14
24747,1.71,Premium,I,VS1,60.7,60.0,13097,7.74,7.71,4.69
22986,2.0,Good,J,SI2,61.5,61.0,11036,7.97,8.06,4.93
27421,2.32,Fair,H,SI1,62.0,62.0,18026,8.47,8.31,5.2
26081,2.0,Very Good,H,SI2,59.7,61.0,15312,8.15,8.2,4.88
21099,1.73,Premium,J,SI1,60.7,58.0,9271,7.78,7.73,4.71
24148,2.3,Ideal,J,SI1,62.3,57.0,12316,8.41,8.34,5.22
25882,2.06,Premium,I,SI2,60.1,58.0,14982,8.32,8.26,4.98
25883,2.01,Ideal,H,SI2,62.5,53.9,14998,8.04,8.07,5.04
26611,2.05,Premium,G,SI2,60.1,59.0,16357,8.2,8.3,4.96
26458,2.02,Premium,H,SI2,59.9,55.0,15996,8.28,8.17,4.93
20983,1.71,Premium,H,SI1,58.1,59.0,9193,7.88,7.81,4.56
22389,2.02,Ideal,I,SI2,62.2,57.0,10412,8.06,7.99,4.99
27090,2.15,Premium,H,SI2,62.8,58.0,17221,8.22,8.17,5.15
26063,1.77,Premium,E,VS2,61.6,58.0,15278,7.78,7.71,4.77
26617,2.28,Premium,J,VS2,62.4,58.0,16369,8.45,8.35,5.24
21815,1.75,Ideal,J,VS2,62.1,56.0,9890,7.74,7.69,4.79
24887,2.06,Premium,G,SI1,59.3,61.0,13317,8.44,8.36,4.98
26079,2.04,Ideal,I,SI1,60.0,60.0,15308,8.3,8.26,4.97
24966,2.02,Premium,H,SI1,63.0,60.0,13453,7.85,7.79,4.93
"""
diamonds = pd.read_csv(io.StringIO(data_str))

# Create "bump" dummies
diamonds["carat_05"] = (diamonds["carat"] >= 0.5).astype(int)
diamonds["carat_10"] = (diamonds["carat"] >= 1.0).astype(int)
diamonds["carat_15"] = (diamonds["carat"] >= 1.5).astype(int)
diamonds["carat_20"] = (diamonds["carat"] >= 2.0).astype(int)

# Convert categorical variables to category dtype
for col in ["cut", "color", "clarity"]:
    diamonds[col] = diamonds[col].astype("category")

# --------------------------------------------------------------------
# 2) Build the Shiny UI
# --------------------------------------------------------------------

app_ui = ui.page_fluid(
    ui.h2("Diamond Price Model with Threshold Bumps & Heteroskedastic Errors"),
    ui.layout_sidebar(
        ui.sidebar(
            ui.input_action_button("fit_model", "Fit/Re-fit Model"),
            ui.input_checkbox("show_fit", "Show Fitted Regression Line", True),
            ui.input_checkbox("show_data", "Show Original Data Points", True),
            ui.input_numeric("carat_value", "Predict Price at Carat", value=1.0, step=0.1),
            width=3
        ),
        ui.output_plot("diamondPlot", height="500px"),
        ui.output_text_verbatim("model_summary")
    )
)


# --------------------------------------------------------------------
# 3) Define the server logic
# --------------------------------------------------------------------
def server(input, output, session):
    # Initialize model storage
    model_store = reactive.Value(None)
    
    @reactive.Effect
    @reactive.event(input.fit_model)
    def fit_wls_model():
        try:
            df = diamonds.copy()
            
            formula = ("price ~ carat + carat_05 + carat_10 + carat_15 + carat_20 + "
                      "C(cut) + C(color) + C(clarity)")
            
            y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')
            
            # Initial OLS
            ols_model = sm.OLS(y, X).fit()
            df["price_pred_ols"] = ols_model.predict(X)
            
            # Weighted LS
            w = 1.0 / np.maximum(df["price_pred_ols"], 1) ** 2
            wls_model = sm.WLS(y, X, weights=w).fit()
            
            # Store the model
            model_store.set({
                'model': wls_model,
                'X': X,
                'fitted_data': df
            })
        except Exception as e:
            print(f"Error fitting model: {str(e)}")

    @output
    @render.plot
    def diamondPlot():
        fig, ax = plt.subplots(figsize=(9, 6))
        
        if input.show_data():
            ax.scatter(diamonds["carat"], diamonds["price"],
                      label="Data", alpha=0.6, color='blue')
        
        if input.show_fit() and model_store() is not None:
            try:
                model = model_store()['model']
                
                carat_vals = np.linspace(diamonds["carat"].min(), 
                                       diamonds["carat"].max(), 100)
                plot_df = pd.DataFrame({
                    "carat": carat_vals,
                    "carat_05": (carat_vals >= 0.5).astype(int),
                    "carat_10": (carat_vals >= 1.0).astype(int),
                    "carat_15": (carat_vals >= 1.5).astype(int),
                    "carat_20": (carat_vals >= 2.0).astype(int),
                    "cut": diamonds["cut"].cat.categories[0],
                    "color": diamonds["color"].cat.categories[0],
                    "clarity": diamonds["clarity"].cat.categories[0]
                })
                
                _, X_plot = patsy.dmatrices(
                    model.model.formula, data=plot_df, return_type='dataframe'
                )
                y_pred = model.predict(X_plot)
                
                ax.plot(carat_vals, y_pred, color='red',
                       label="Fitted (Weighted LS)")
            
            except Exception as e:
                print(f"Error plotting fitted line: {str(e)}")
        
        ax.set_xlabel("Carat")
        ax.set_ylabel("Price (USD)")
        ax.set_title("Diamond Price vs. Carat with Threshold Bumps & Heteroskedastic Errors")
        ax.legend()
        ax.grid(True)
        
        return fig

    @output
    @render.text
    def model_summary():
        if model_store() is not None:
            return model_store()['model'].summary().as_text()
        return "No model fitted yet. Click 'Fit/Re-fit Model'"

app = App(app_ui, server)
```

2nd try:

```{shinylive-python}
#| standalone: true
#| viewerHeight: 650

from shiny import App, ui, render, reactive
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import io

# Set the matplotlib backend explicitly
plt.switch_backend('agg')

# Just including the data in the code for simplicity
# It is a stratified sample based on carats
data_str = """
ID,carat,cut,color,clarity,depth,table,price,x,y,z
51657,0.3,Ideal,G,VS2,62.3,58.0,545,4.26,4.28,2.66
34838,0.3,Premium,G,VVS2,60.8,58.0,878,4.38,4.34,2.65
9718,0.3,Ideal,H,VVS2,62.1,54.0,590,4.32,4.35,2.69
46635,0.3,Very Good,E,SI1,62.7,60.0,526,4.24,4.28,2.67
31852,0.3,Premium,G,VS1,62.2,59.0,776,4.28,4.24,2.65
40942,0.27,Ideal,H,VS1,62.3,54.0,500,4.16,4.19,2.6
49960,0.3,Good,H,SI1,63.7,56.0,540,4.22,4.2,2.68
30300,0.3,Very Good,D,SI2,61.0,61.0,447,4.25,4.31,2.61
15051,0.3,Ideal,F,VS2,61.4,57.0,605,4.34,4.36,2.67
32272,0.3,Very Good,G,VVS1,62.9,57.0,789,4.26,4.3,2.69
16695,0.3,Very Good,H,SI1,62.6,58.0,421,4.22,4.28,2.66
32358,0.3,Good,G,VVS1,63.1,56.0,789,4.25,4.28,2.69
3393,0.27,Very Good,E,VVS2,59.4,64.0,567,4.16,4.19,2.48
16027,0.3,Premium,I,VS1,60.5,60.0,608,4.33,4.3,2.61
5721,0.25,Very Good,E,VVS2,60.9,59.0,575,4.03,4.11,2.48
34695,0.3,Ideal,F,IF,61.7,56.0,873,4.31,4.35,2.67
28794,0.27,Very Good,F,VVS2,61.3,57.0,682,4.14,4.18,2.54
32496,0.3,Good,F,IF,58.8,61.0,796,4.35,4.39,2.57
16359,0.3,Good,D,VS2,64.1,57.0,608,4.25,4.21,2.71
31973,0.3,Very Good,I,VS2,60.5,55.0,453,4.34,4.37,2.63
51312,0.31,Ideal,G,VS2,59.1,57.0,544,4.45,4.48,2.64
27844,0.31,Very Good,G,VS2,63.2,58.0,651,4.3,4.28,2.71
37309,0.31,Ideal,F,IF,62.2,56.0,979,4.31,4.34,2.69
16685,0.31,Ideal,H,SI2,61.1,56.0,421,4.4,4.42,2.69
35803,0.31,Premium,F,IF,61.9,58.0,914,4.36,4.39,2.71
30256,0.31,Very Good,E,VVS1,60.4,61.0,725,4.34,4.4,2.64
36008,0.31,Ideal,F,IF,61.2,56.0,921,4.37,4.42,2.69
30803,0.31,Good,F,VVS1,63.6,61.0,742,4.21,4.25,2.69
32676,0.31,Premium,G,VS1,62.4,59.0,802,4.34,4.32,2.7
35593,0.31,Ideal,H,VVS1,62.2,54.0,907,4.39,4.36,2.72
20386,0.31,Premium,G,VS1,59.5,59.0,625,4.4,4.47,2.64
34570,0.31,Ideal,G,IF,61.0,55.0,871,4.39,4.42,2.69
33609,0.31,Ideal,D,SI2,62.0,56.0,462,4.33,4.35,2.69
32609,0.31,Premium,H,VVS2,61.4,59.0,802,4.38,4.35,2.68
32723,0.31,Ideal,F,VS2,62.7,57.0,802,4.34,4.3,2.71
44998,0.31,Premium,I,SI1,62.3,59.0,523,4.32,4.29,2.68
38803,0.31,Very Good,G,VVS1,63.1,56.0,1046,4.35,4.33,2.74
43285,0.31,Very Good,D,SI1,60.4,60.0,507,4.4,4.44,2.67
33131,0.31,Very Good,E,VVS2,60.8,55.0,816,4.38,4.43,2.68
35157,0.31,Very Good,G,IF,61.6,54.0,891,4.4,4.43,2.72
37580,0.32,Premium,D,VVS2,61.5,60.0,990,4.41,4.37,2.7
33506,0.32,Premium,G,VS1,62.5,60.0,828,4.35,4.29,2.7
26341,0.32,Ideal,H,VVS2,61.7,56.0,645,4.37,4.42,2.71
33033,0.32,Ideal,G,VVS1,61.4,57.0,814,4.39,4.41,2.7
36290,0.32,Ideal,G,SI1,61.3,57.0,477,4.37,4.4,2.69
36284,0.32,Ideal,D,SI2,62.4,54.0,477,4.38,4.4,2.74
13404,0.32,Very Good,F,VS2,61.2,58.0,602,4.38,4.41,2.69
30954,0.32,Ideal,I,VS2,62.5,55.0,449,4.38,4.39,2.74
29634,0.32,Ideal,J,VS1,62.0,54.7,442,4.39,4.42,2.73
30129,0.32,Ideal,G,VS2,61.8,57.0,720,4.4,4.37,2.71
46963,0.32,Good,F,SI1,61.6,60.1,528,4.38,4.4,2.71
32783,0.32,Ideal,D,VVS2,61.2,56.0,803,4.39,4.43,2.7
20012,0.32,Good,G,SI2,63.4,55.0,421,4.32,4.35,2.75
34133,0.32,Ideal,F,VVS1,60.4,57.0,854,4.41,4.43,2.67
27865,0.32,Ideal,G,SI1,61.4,56.0,653,4.44,4.42,2.72
29989,0.32,Ideal,F,VS1,61.0,54.0,716,4.42,4.44,2.7
30145,0.32,Premium,G,VS2,62.8,58.0,720,4.35,4.31,2.72
31320,0.32,Ideal,D,VS2,62.6,55.0,758,4.37,4.39,2.74
35896,0.32,Ideal,G,IF,61.7,54.0,918,4.42,4.46,2.74
50304,0.32,Very Good,G,VS2,62.3,55.0,544,4.38,4.41,2.73
32501,0.33,Premium,G,VS1,61.6,57.0,797,4.51,4.42,2.75
29919,0.33,Ideal,H,VVS1,61.8,55.0,713,4.42,4.44,2.74
37434,0.33,Good,G,IF,57.9,60.0,984,4.55,4.57,2.64
42419,0.33,Ideal,E,VVS1,61.9,57.0,1312,4.43,4.46,2.75
30338,0.34,Premium,F,SI1,59.4,62.0,727,4.59,4.54,2.71
23380,0.33,Very Good,G,SI1,63.2,57.0,631,4.44,4.39,2.79
18704,0.35,Very Good,I,VVS2,61.3,56.0,620,4.52,4.54,2.78
31350,0.34,Ideal,E,VS2,61.8,54.0,760,4.49,4.5,2.78
34543,0.35,Ideal,H,IF,61.5,57.0,868,4.55,4.58,2.8
13389,0.35,Premium,D,SI1,61.5,58.0,601,4.53,4.55,2.79
36970,0.34,Ideal,D,VS1,60.7,57.0,961,4.55,4.51,2.75
37025,0.33,Ideal,G,VVS2,62.5,54.0,965,4.45,4.41,2.77
30831,0.33,Premium,I,VVS2,61.5,58.0,743,4.45,4.43,2.73
36287,0.34,Very Good,E,SI2,61.7,61.0,477,4.47,4.51,2.77
34161,0.33,Premium,G,VS1,60.5,58.0,854,4.49,4.43,2.7
30719,0.35,Fair,E,VVS2,66.2,61.0,738,4.4,4.36,2.9
33204,0.35,Ideal,G,VVS2,61.8,55.0,820,4.53,4.56,2.81
26014,0.35,Premium,D,SI1,60.9,58.0,644,4.52,4.55,2.76
27052,0.33,Ideal,I,VVS1,62.2,54.0,646,4.43,4.45,2.76
34181,0.33,Ideal,G,VS1,62.1,56.0,854,4.42,4.4,2.74
28218,0.4,Premium,D,SI2,62.1,60.0,666,4.69,4.75,2.93
39564,0.4,Premium,G,VS1,62.2,55.0,1080,4.83,4.69,2.96
33662,0.36,Ideal,E,VS1,61.4,54.0,835,4.59,4.63,2.83
36552,0.4,Ideal,E,SI1,60.5,57.0,945,4.81,4.77,2.9
37369,0.4,Very Good,F,VS1,60.4,61.0,982,4.74,4.77,2.87
41873,0.38,Ideal,D,VVS2,61.5,56.0,1257,4.66,4.64,2.86
35767,0.4,Premium,E,VS2,60.7,60.0,912,4.7,4.75,2.87
27792,0.37,Premium,G,VS2,61.3,60.0,649,4.6,4.63,2.83
41652,0.4,Ideal,E,VVS2,62.1,56.0,1238,4.73,4.7,2.93
37757,0.38,Premium,D,VS2,61.6,59.0,998,4.66,4.62,2.86
36266,0.37,Ideal,H,IF,61.7,53.0,936,4.66,4.68,2.88
37328,0.4,Premium,G,VVS2,61.3,59.0,980,4.78,4.74,2.92
38250,0.36,Ideal,D,VS1,62.8,55.0,1018,4.55,4.52,2.85
35397,0.38,Good,F,VS2,62.4,54.3,899,4.6,4.65,2.89
31021,0.37,Premium,I,VS1,61.4,59.0,749,4.61,4.55,2.81
30667,0.4,Very Good,I,VS1,63.0,56.0,737,4.68,4.72,2.96
39618,0.37,Very Good,H,SI1,62.6,63.0,491,4.6,4.5,2.85
30669,0.4,Premium,F,SI1,62.5,59.0,737,4.67,4.71,2.93
17728,0.39,Ideal,E,SI2,61.0,55.0,614,4.74,4.77,2.9
35328,0.38,Ideal,H,VVS2,62.1,54.0,898,4.62,4.66,2.88
33367,0.41,Ideal,G,VS2,61.4,55.0,827,4.75,4.8,2.93
39486,0.41,Ideal,E,VS1,62.1,55.0,1079,4.75,4.78,2.96
31789,0.42,Ideal,E,SI1,61.3,57.0,773,4.79,4.81,2.94
33930,0.41,Good,G,VVS1,63.6,56.0,844,4.72,4.74,3.01
41724,0.41,Ideal,H,IF,61.8,55.0,1243,4.79,4.76,2.95
42168,0.41,Premium,D,VS1,59.3,58.0,1286,4.87,4.85,2.88
30052,0.41,Premium,G,SI1,59.1,58.0,719,4.83,4.88,2.87
41467,0.41,Premium,G,VVS1,61.0,61.0,1230,4.75,4.72,2.89
35509,0.41,Premium,E,SI1,62.8,58.0,904,4.77,4.72,2.98
24390,0.41,Very Good,E,SI2,63.0,57.0,638,4.7,4.73,2.97
35351,0.42,Ideal,H,SI1,62.4,57.0,898,4.79,4.76,2.98
37077,0.41,Premium,F,SI1,62.6,55.0,969,4.78,4.74,2.98
36978,0.42,Premium,G,VVS2,61.6,60.0,963,4.8,4.85,2.97
28454,0.41,Ideal,G,SI1,62.2,56.0,671,4.75,4.77,2.96
43252,0.42,Premium,G,IF,60.2,59.0,1400,4.8,4.87,2.91
41015,0.41,Very Good,F,VVS1,62.7,59.0,1186,4.75,4.78,2.99
37665,0.42,Premium,E,SI1,61.6,59.0,992,4.85,4.83,2.98
40213,0.41,Ideal,D,SI1,61.8,56.0,1122,4.78,4.73,2.94
37909,0.41,Ideal,F,VS1,60.8,56.0,1007,4.76,4.79,2.92
39436,0.41,Ideal,D,VS2,62.2,54.0,1076,4.81,4.77,2.98
46182,0.5,Ideal,I,VVS1,61.6,56.0,1747,5.1,5.13,3.15
38815,0.45,Premium,F,SI1,61.1,58.0,1046,4.97,4.95,3.03
41423,0.46,Ideal,H,VVS1,62.3,54.0,1227,4.96,4.99,3.1
50341,0.5,Ideal,D,VS2,61.1,57.0,2243,5.11,5.13,3.13
43455,0.5,Premium,G,VS2,61.5,57.0,1415,5.12,5.09,3.14
35239,0.43,Very Good,E,SI1,63.4,56.0,894,4.82,4.8,3.05
41838,0.44,Ideal,F,VVS2,60.9,55.0,1253,4.96,4.92,3.01
37303,0.5,Premium,G,SI2,60.7,57.0,978,5.15,5.07,3.1
37391,0.5,Ideal,I,SI1,62.0,55.0,982,5.08,5.11,3.16
38196,0.5,Very Good,D,SI2,63.1,56.0,1015,5.05,4.96,3.16
33009,0.43,Premium,F,SI2,58.3,62.0,813,4.97,4.91,2.88
43403,0.46,Ideal,G,VVS1,62.0,54.0,1412,4.97,5.0,3.09
44797,0.5,Very Good,E,VS2,61.5,56.0,1624,5.07,5.11,3.13
32446,0.43,Very Good,H,VS2,61.9,55.0,792,4.8,4.95,3.02
39507,0.5,Ideal,F,SI2,61.7,55.0,1080,5.13,5.15,3.17
42348,0.46,Ideal,H,SI1,61.2,56.0,1299,4.97,5.0,3.05
49157,0.5,Very Good,G,VVS1,63.3,56.0,2070,5.1,5.07,3.22
39697,0.48,Good,G,VS2,65.4,59.0,1088,4.79,4.88,3.16
47045,0.5,Premium,D,VS2,59.7,57.0,1819,5.13,5.08,3.05
38353,0.47,Very Good,F,SI1,61.1,61.0,1021,4.97,5.01,3.05
49694,0.51,Very Good,E,VVS2,62.8,57.0,2146,5.06,5.1,3.19
39316,0.53,Very Good,G,SI2,60.8,58.0,1070,5.19,5.21,3.16
44608,0.53,Premium,E,SI1,61.9,56.0,1607,5.22,5.19,3.22
47613,0.53,Ideal,G,VVS2,60.4,55.0,1881,5.26,5.3,3.19
44575,0.53,Ideal,E,VS2,62.5,57.0,1607,5.16,5.18,3.23
49934,0.51,Premium,E,VVS2,62.1,57.0,2185,5.18,5.15,3.21
41199,0.51,Very Good,D,SI2,60.3,57.0,1204,5.15,5.17,3.11
47601,0.52,Ideal,G,VVS2,60.8,57.0,1878,5.2,5.17,3.15
48545,0.52,Ideal,I,IF,60.2,56.0,1988,5.23,5.27,3.16
41422,0.52,Very Good,F,SI1,62.3,55.0,1227,5.14,5.17,3.21
48904,0.51,Very Good,F,VVS2,62.0,56.0,2041,5.1,5.15,3.17
43201,0.53,Good,G,VS2,63.4,58.0,1395,5.13,5.16,3.26
46534,0.51,Ideal,G,VS1,62.5,57.0,1781,5.14,5.07,3.19
43116,0.52,Very Good,H,VS2,63.5,58.0,1385,5.12,5.11,3.25
36885,0.51,Good,I,SI1,63.1,56.0,959,5.06,5.14,3.22
44284,0.51,Ideal,G,VS1,62.5,57.0,1577,5.08,5.1,3.18
37127,0.52,Ideal,D,I1,61.1,57.0,971,5.18,5.2,3.17
48116,0.52,Ideal,G,VVS1,61.9,54.4,1936,5.15,5.18,3.2
44258,0.51,Ideal,H,VVS2,61.0,57.0,1574,5.22,5.18,3.17
46475,0.51,Ideal,H,VVS1,61.4,55.0,1776,5.13,5.16,3.16
46460,0.54,Ideal,F,VS1,61.1,57.0,1774,5.28,5.3,3.23
50067,0.54,Ideal,F,VS1,61.5,55.0,2202,5.26,5.27,3.24
43563,0.58,Fair,G,VS2,65.0,56.0,1430,5.23,5.17,3.38
47010,0.56,Ideal,E,VS2,60.9,56.0,1819,5.32,5.35,3.25
41886,0.54,Ideal,I,VS2,61.1,55.0,1259,5.27,5.31,3.23
42007,0.59,Ideal,F,SI2,61.8,55.0,1265,5.41,5.44,3.35
48843,0.55,Ideal,E,VS2,62.5,56.0,2030,5.26,5.23,3.28
52201,0.54,Ideal,E,VVS2,61.9,54.5,2479,5.22,5.25,3.23
49498,0.56,Ideal,H,VVS2,61.8,56.0,2118,5.28,5.33,3.28
52348,0.55,Ideal,E,VVS2,61.4,56.0,2499,5.28,5.31,3.25
50508,0.54,Ideal,G,IF,62.3,56.0,2271,5.19,5.21,3.24
46004,0.54,Ideal,D,VS2,61.2,56.0,1725,5.24,5.28,3.22
46440,0.54,Ideal,F,VS1,60.9,57.0,1772,5.21,5.26,3.19
45822,0.56,Good,F,VS1,63.2,61.0,1712,5.2,5.28,3.3
46373,0.58,Ideal,G,VS2,61.9,55.0,1761,5.33,5.36,3.31
41799,0.6,Very Good,E,SI2,63.2,60.0,1250,5.32,5.28,3.35
45126,0.59,Very Good,E,SI1,62.9,58.0,1652,5.31,5.34,3.35
43185,0.54,Very Good,G,SI1,63.2,58.0,1392,5.15,5.16,3.26
45719,0.56,Ideal,E,SI1,62.7,57.0,1698,5.27,5.23,3.29
42200,0.56,Premium,G,SI1,61.1,61.0,1287,5.31,5.29,3.24
3262,0.7,Ideal,F,VS1,60.3,57.0,3359,5.74,5.79,3.47
51331,0.7,Very Good,F,VS2,62.3,56.0,2362,5.66,5.71,3.54
50892,0.7,Premium,G,VS2,60.8,58.0,2317,5.75,5.8,3.51
46073,0.63,Premium,F,SI1,59.1,57.0,1736,5.64,5.6,3.32
53792,0.7,Very Good,E,SI1,62.1,60.0,2730,5.62,5.66,3.5
1543,0.7,Very Good,D,VS1,63.4,59.0,3001,5.58,5.55,3.53
2516,0.7,Ideal,E,VS2,60.5,59.0,3201,5.72,5.75,3.47
52766,0.7,Very Good,G,VS2,58.7,53.0,2563,5.83,5.86,3.43
52504,0.7,Good,D,SI1,58.0,60.0,2525,5.79,5.93,3.4
52161,0.7,Premium,D,SI1,60.8,58.0,2473,5.79,5.66,3.48
44158,0.7,Fair,F,SI2,66.4,56.0,1564,5.51,5.42,3.63
46845,0.64,Premium,E,SI1,61.3,58.0,1811,5.57,5.53,3.4
47260,0.7,Premium,J,VS2,61.2,60.0,1843,5.7,5.73,3.5
2424,0.63,Ideal,E,VVS1,61.1,58.0,3181,5.49,5.54,3.37
48887,0.7,Very Good,F,SI2,59.6,61.0,2039,5.8,5.88,3.48
51599,0.7,Good,I,VVS2,63.3,55.0,2394,5.61,5.67,3.57
46198,0.7,Fair,I,SI1,65.2,58.0,1749,5.6,5.56,3.64
49877,0.7,Premium,H,SI1,60.9,62.0,2176,5.72,5.67,3.47
52012,0.7,Good,D,SI1,59.9,63.0,2444,5.74,5.81,3.46
2986,0.7,Ideal,G,VS1,60.8,56.0,3300,5.73,5.8,3.51
277,0.71,Very Good,E,VS2,60.7,56.0,2795,5.81,5.82,3.53
809,0.71,Premium,D,SI1,59.7,59.0,2863,5.82,5.8,3.47
52887,0.72,Premium,H,VS2,60.7,59.0,2583,5.84,5.8,3.53
946,0.72,Very Good,G,VVS2,62.5,58.0,2889,5.68,5.72,3.56
51695,0.71,Very Good,I,VVS2,59.5,60.0,2400,5.82,5.87,3.48
48158,0.72,Very Good,H,SI2,63.5,58.0,1942,5.65,5.68,3.6
51672,0.72,Ideal,E,SI2,61.9,55.0,2398,5.76,5.78,3.57
3806,0.72,Ideal,E,VS1,62.5,57.0,3465,5.73,5.76,3.59
51150,0.71,Premium,F,SI2,62.0,59.0,2343,5.68,5.65,3.51
694,0.71,Premium,F,VS2,62.6,58.0,2853,5.67,5.7,3.56
50848,0.72,Premium,H,SI1,62.2,57.0,2311,5.75,5.72,3.57
45878,0.71,Premium,G,SI2,59.9,59.0,1717,5.79,5.82,3.48
49717,0.72,Premium,I,SI1,61.5,59.0,2148,5.73,5.78,3.54
2140,0.72,Ideal,H,VVS1,61.4,56.0,3124,5.79,5.77,3.55
1181,0.71,Ideal,G,VS1,62.7,57.0,2930,5.69,5.73,3.58
50722,0.71,Premium,I,VS2,62.1,59.0,2294,5.7,5.73,3.55
53191,0.71,Premium,F,SI1,62.7,57.0,2633,5.68,5.65,3.55
48876,0.71,Very Good,F,SI2,63.3,56.0,2036,5.68,5.73,3.61
3635,0.71,Ideal,G,VS1,60.7,57.0,3431,5.76,5.8,3.51
51843,0.71,Very Good,E,SI2,62.2,58.0,2423,5.65,5.7,3.53
53670,0.74,Very Good,H,VS1,61.9,59.1,2709,5.74,5.77,3.56
7260,0.9,Ideal,F,SI2,61.5,56.0,4198,6.24,6.18,3.82
7909,0.9,Ideal,G,SI2,60.7,57.0,4314,6.19,6.33,3.8
8568,0.9,Premium,F,SI1,61.4,55.0,4435,6.18,6.16,3.79
1110,0.8,Very Good,F,SI1,63.5,55.0,2914,5.86,5.89,3.73
53096,0.75,Ideal,I,VS1,63.0,57.0,2613,5.8,5.82,3.66
1207,0.76,Premium,E,SI1,58.3,62.0,2937,6.12,5.95,3.52
580,0.78,Ideal,I,VS2,61.8,55.0,2834,5.92,5.95,3.67
47891,0.74,Very Good,J,SI1,62.2,59.0,1913,5.74,5.81,3.59
1486,0.77,Premium,E,SI1,61.7,58.0,2988,5.86,5.9,3.63
53472,0.76,Ideal,E,SI2,61.5,55.0,2680,5.88,5.93,3.63
4245,0.84,Good,E,SI1,61.9,61.0,3577,6.03,6.05,3.74
4671,0.76,Ideal,G,VVS1,62.0,54.7,3671,5.83,5.87,3.62
1813,0.78,Very Good,E,SI1,60.9,57.0,3055,5.93,5.97,3.62
682,0.75,Ideal,J,SI1,61.5,56.0,2850,5.83,5.87,3.6
113,0.9,Premium,I,VS2,63.0,58.0,2761,6.16,6.12,3.87
3221,0.9,Very Good,G,SI2,63.5,57.0,3350,6.09,6.13,3.88
9439,0.9,Very Good,H,VVS2,63.7,57.0,4592,6.09,6.02,3.86
53398,0.83,Ideal,H,SI2,61.1,59.0,2666,6.05,6.1,3.71
4108,0.74,Ideal,G,VVS1,62.1,54.0,3537,5.8,5.83,3.61
4215,0.91,Very Good,H,VS2,63.1,56.0,3567,6.2,6.13,3.89
9572,1.0,Premium,D,SI2,62.2,61.0,4626,6.36,6.3,3.94
8097,0.95,Premium,D,SI2,60.1,61.0,4341,6.37,6.35,3.82
14644,1.0,Premium,H,VVS2,61.4,59.0,5914,6.49,6.45,3.97
12007,1.0,Good,G,VS2,63.8,59.0,5148,6.26,6.34,4.02
3802,1.0,Very Good,J,SI1,61.9,62.0,3465,6.33,6.36,3.93
6503,0.97,Fair,F,SI1,56.4,66.0,4063,6.59,6.54,3.7
9575,1.0,Premium,D,SI2,59.4,60.0,4626,6.56,6.48,3.87
4748,0.92,Premium,F,SI1,62.6,59.0,3684,6.23,6.19,3.89
10565,1.0,Premium,G,SI1,60.8,58.0,4816,6.48,6.45,3.93
9806,0.91,Very Good,E,SI2,63.2,56.0,4668,6.08,6.14,3.86
13270,1.0,Good,G,VS2,56.6,61.0,5484,6.65,6.61,3.75
18435,1.0,Good,D,VS1,57.8,61.0,7500,6.62,6.56,3.81
3591,0.91,Premium,G,SI2,61.3,60.0,3423,6.17,6.2,3.79
5447,1.0,Fair,H,SI1,55.2,64.0,3830,6.69,6.64,3.68
15947,1.0,Premium,G,VS1,62.4,60.0,6377,6.39,6.37,3.98
10800,1.0,Good,H,VS2,63.7,59.0,4861,6.3,6.26,4.0
5849,1.0,Premium,H,SI2,61.3,58.0,3920,6.45,6.41,3.94
8315,0.91,Very Good,D,SI1,63.5,56.0,4389,6.13,6.18,3.91
4151,0.91,Premium,F,SI2,61.0,51.0,3546,6.24,6.21,3.8
9426,1.01,Very Good,D,SI2,62.8,59.0,4588,6.34,6.44,4.01
10581,1.01,Very Good,D,SI1,59.1,61.0,4821,6.46,6.5,3.83
15174,1.01,Very Good,H,VVS2,63.3,57.0,6097,6.39,6.35,4.03
5937,1.01,Very Good,F,SI2,60.8,63.0,3945,6.32,6.38,3.86
9236,1.01,Good,H,SI1,63.3,58.0,4559,6.37,6.4,4.04
15117,1.01,Premium,D,SI1,61.8,58.0,6075,6.42,6.37,3.95
7700,1.01,Fair,F,SI1,67.2,60.0,4276,6.06,6.0,4.05
9013,1.01,Premium,H,SI1,61.3,58.0,4513,6.47,6.39,3.94
15740,1.01,Ideal,G,VS2,60.6,58.0,6295,6.44,6.5,3.92
11337,1.01,Good,F,SI1,63.7,57.0,4989,6.4,6.35,4.06
15199,1.01,Very Good,G,VS2,61.9,56.0,6105,6.34,6.42,3.95
10942,1.01,Very Good,F,SI1,59.7,61.0,4899,6.49,6.55,3.89
4744,1.01,Very Good,G,SI2,62.0,58.0,3682,6.41,6.46,3.99
18733,1.01,Very Good,D,VS2,62.7,57.0,7652,6.36,6.39,4.0
15525,1.01,Very Good,E,VS2,63.0,60.0,6221,6.32,6.35,3.99
16288,1.01,Very Good,E,VS2,63.3,60.0,6516,6.33,6.3,4.0
11015,1.01,Very Good,G,SI1,60.6,57.0,4916,6.49,6.52,3.94
16798,1.01,Premium,E,VS2,60.4,57.0,6697,6.49,6.45,3.91
11293,1.01,Ideal,H,SI1,62.3,55.0,4977,6.43,6.37,3.99
13505,1.01,Ideal,D,SI1,61.2,57.0,5543,6.47,6.44,3.95
13562,1.02,Very Good,E,SI1,59.2,56.0,5553,6.57,6.63,3.91
9083,1.03,Premium,E,SI2,61.0,60.0,4522,6.53,6.46,3.96
9159,1.02,Very Good,E,SI2,63.3,58.0,4540,6.31,6.4,4.02
10316,1.03,Very Good,G,SI1,63.2,58.0,4764,6.43,6.38,4.05
12600,1.02,Very Good,F,SI1,60.9,57.0,5287,6.52,6.56,3.98
15398,1.02,Very Good,G,VS2,63.4,59.0,6169,6.32,6.3,4.0
8405,1.03,Ideal,I,SI1,63.3,57.0,4401,6.37,6.46,4.06
17889,1.04,Ideal,D,VS2,61.9,55.0,7220,6.5,6.52,4.03
7153,1.04,Very Good,F,SI2,62.3,58.0,4181,6.44,6.5,4.03
16983,1.03,Premium,F,VS1,61.7,56.0,6783,6.49,6.47,4.0
11198,1.02,Premium,H,VS2,60.0,58.0,4958,6.56,6.5,3.92
5865,1.03,Ideal,J,SI1,62.6,57.0,3922,6.45,6.43,4.03
15016,1.02,Very Good,D,SI1,62.8,56.0,6047,6.39,6.44,4.03
7502,1.04,Premium,E,SI2,61.6,59.0,4240,6.57,6.55,4.04
14328,1.03,Ideal,D,SI1,61.2,55.0,5804,6.51,6.57,4.0
8632,1.02,Premium,G,SI1,62.6,59.0,4449,6.43,6.38,4.01
7041,1.02,Ideal,F,SI2,62.1,56.0,4162,6.41,6.44,3.99
21809,1.03,Ideal,F,VVS1,61.3,54.0,9881,6.56,6.62,4.04
48885,1.04,Fair,I,I1,67.3,56.0,2037,6.34,6.23,4.22
16635,1.02,Premium,F,VS2,62.4,59.0,6652,6.4,6.45,4.01
15538,1.09,Ideal,I,VS1,61.8,55.0,6225,6.59,6.62,4.08
18682,1.11,Ideal,G,VS1,61.5,58.0,7639,6.7,6.66,4.11
7580,1.06,Very Good,I,SI1,62.8,56.0,4255,6.47,6.52,4.08
8646,1.06,Premium,F,SI2,62.4,58.0,4452,6.54,6.5,4.07
20512,1.11,Ideal,G,VVS2,63.1,57.0,8843,6.55,6.6,4.15
13460,1.13,Very Good,G,SI1,63.1,58.0,5526,6.65,6.59,4.18
11822,1.07,Ideal,I,SI1,61.7,56.0,5093,6.59,6.57,4.06
19907,1.09,Premium,G,VVS2,59.5,61.0,8454,6.74,6.7,4.0
16948,1.08,Ideal,G,VS2,60.3,59.0,6769,6.62,6.64,4.0
15439,1.05,Premium,G,VS2,61.8,58.0,6181,6.59,6.52,4.05
17304,1.09,Ideal,G,VS1,62.4,57.0,6934,6.55,6.63,4.11
14807,1.11,Ideal,E,SI2,60.6,56.0,5962,6.76,6.78,4.1
21425,1.07,Ideal,G,IF,61.5,57.0,9532,6.59,6.54,4.04
4661,1.13,Ideal,H,I1,61.1,56.0,3669,6.77,6.71,4.12
16344,1.1,Ideal,G,VS1,61.3,54.0,6535,6.69,6.65,4.09
11847,1.05,Ideal,I,VS1,61.5,55.0,5101,6.56,6.61,4.05
16867,1.07,Premium,G,VS1,62.0,58.0,6730,6.59,6.53,4.07
21535,1.12,Ideal,F,VVS2,61.4,57.0,9634,6.69,6.66,4.1
8220,1.09,Very Good,J,VS2,62.3,59.0,4372,6.56,6.63,4.11
18833,1.12,Ideal,G,VS1,61.6,55.0,7716,6.69,6.72,4.13
13956,1.16,Very Good,G,SI1,60.7,59.0,5678,6.74,6.87,4.13
20531,1.23,Premium,F,VS2,59.6,58.0,8855,6.94,7.02,4.16
12498,1.15,Very Good,E,SI2,60.0,59.0,5257,6.78,6.82,4.08
14003,1.2,Premium,I,VS2,62.6,58.0,5699,6.77,6.72,4.22
22973,1.2,Premium,F,VVS2,62.2,58.0,11021,6.83,6.78,4.23
8795,1.21,Premium,F,SI2,61.8,59.0,4472,6.82,6.77,4.2
18812,1.24,Ideal,H,VS2,60.1,59.0,7701,6.99,7.03,4.21
26565,1.2,Ideal,E,VVS1,61.8,56.0,16256,6.78,6.87,4.22
20122,1.24,Ideal,G,VS1,61.9,54.0,8584,6.89,6.92,4.27
12313,1.24,Ideal,I,SI2,61.9,57.0,5221,6.87,6.92,4.27
15155,1.21,Premium,F,SI2,59.0,60.0,6092,6.99,6.94,4.11
18869,1.22,Ideal,H,VS1,60.4,57.0,7738,6.86,6.89,4.15
16067,1.2,Premium,H,VS2,62.5,58.0,6416,6.77,6.73,4.23
10468,1.21,Very Good,I,SI2,62.1,59.0,4791,6.8,6.86,4.24
12328,1.2,Very Good,J,VS1,62.9,60.0,5226,6.64,6.69,4.19
7885,1.21,Premium,F,SI2,62.4,60.0,4310,6.77,6.73,4.21
23561,1.21,Ideal,G,VVS1,61.5,56.0,11572,6.83,6.89,4.22
20700,1.22,Very Good,G,VVS2,61.9,58.0,8975,6.84,6.85,4.24
20006,1.2,Ideal,G,VS1,62.4,57.0,8545,6.78,6.8,4.24
15584,1.2,Premium,F,SI1,62.4,58.0,6250,6.81,6.75,4.23
24545,1.51,Premium,G,VS1,62.4,60.0,12831,7.3,7.34,4.57
26041,1.5,Premium,D,VS2,61.8,60.0,15240,7.37,7.3,4.53
25000,1.5,Very Good,G,VS2,61.1,60.0,13528,7.4,7.3,4.49
6157,1.25,Fair,H,SI2,64.4,58.0,3990,6.82,6.71,4.36
10957,1.25,Ideal,H,SI2,61.6,54.0,4900,6.94,6.88,4.25
14113,1.4,Premium,G,SI2,60.6,58.0,5723,7.26,7.22,4.39
15653,1.26,Ideal,F,SI2,62.7,58.0,6277,6.91,6.87,4.32
12682,1.26,Ideal,J,VS2,63.2,57.0,5306,6.86,6.81,4.32
21426,1.5,Very Good,I,VS2,63.3,55.0,9533,7.3,7.26,4.61
22405,1.5,Good,G,SI1,64.2,58.0,10428,7.14,7.2,4.6
20409,1.5,Premium,F,SI1,62.1,60.0,8770,7.32,7.27,4.53
19944,1.5,Premium,H,SI2,62.3,60.0,8490,7.22,7.3,4.52
16950,1.5,Very Good,H,SI2,63.3,57.0,6770,7.27,7.21,4.59
19527,1.5,Good,I,SI1,62.9,60.0,8161,7.12,7.16,4.49
19250,1.33,Premium,H,VS2,60.7,59.0,7982,7.08,7.13,4.31
15127,1.32,Very Good,J,VS2,62.1,57.0,6079,7.01,7.04,4.36
24098,1.5,Very Good,E,SI1,59.3,60.0,12247,7.4,7.5,4.42
16218,1.33,Very Good,H,SI2,62.5,58.0,6482,7.04,6.97,4.38
20898,1.51,Premium,I,VS2,63.0,60.0,9116,7.3,7.25,4.58
21870,1.25,Ideal,D,VS2,62.6,56.0,9933,6.84,6.87,4.29
25222,1.7,Ideal,H,VS1,62.4,55.0,13823,7.61,7.69,4.77
24230,1.62,Good,H,VS2,61.5,60.8,12429,7.48,7.53,4.62
22614,1.52,Good,F,SI1,63.6,54.0,10664,7.33,7.22,4.63
22933,1.52,Ideal,I,VVS1,61.9,56.0,10968,7.34,7.37,4.55
19386,1.55,Ideal,I,SI2,60.7,60.0,8056,7.49,7.46,4.54
20220,1.54,Premium,J,VVS2,61.1,59.0,8652,7.45,7.4,4.54
24512,1.53,Ideal,E,SI1,62.3,54.2,12791,7.35,7.38,4.59
21122,1.54,Very Good,J,VS1,63.5,57.0,9285,7.27,7.37,4.65
23411,1.67,Premium,I,VS1,61.1,58.0,11400,7.69,7.6,4.67
19348,1.56,Good,I,SI2,58.5,61.0,8048,7.58,7.63,4.45
19758,1.56,Premium,J,VS1,61.1,59.0,8324,7.49,7.52,4.58
25204,1.52,Very Good,D,VS2,62.4,58.0,13799,7.23,7.28,4.53
27338,1.7,Ideal,F,VS2,62.3,56.0,17892,7.61,7.65,4.75
27530,1.7,Ideal,G,VVS1,61.0,56.0,18279,7.62,7.67,4.66
25164,1.7,Premium,F,VS2,62.5,61.0,13737,7.54,7.45,4.69
24018,1.7,Ideal,D,SI1,60.0,54.0,12190,7.76,7.71,4.64
15979,1.7,Ideal,H,I1,61.3,55.0,6397,7.7,7.63,4.7
25184,1.52,Ideal,G,VS2,62.1,56.0,13768,7.39,7.34,4.57
20248,1.55,Ideal,H,SI2,62.1,57.0,8678,7.39,7.43,4.6
17928,1.53,Ideal,G,SI2,61.7,57.0,7240,7.44,7.41,4.58
24211,2.14,Ideal,H,SI2,61.9,57.0,12400,8.34,8.28,5.14
24747,1.71,Premium,I,VS1,60.7,60.0,13097,7.74,7.71,4.69
22986,2.0,Good,J,SI2,61.5,61.0,11036,7.97,8.06,4.93
27421,2.32,Fair,H,SI1,62.0,62.0,18026,8.47,8.31,5.2
26081,2.0,Very Good,H,SI2,59.7,61.0,15312,8.15,8.2,4.88
21099,1.73,Premium,J,SI1,60.7,58.0,9271,7.78,7.73,4.71
24148,2.3,Ideal,J,SI1,62.3,57.0,12316,8.41,8.34,5.22
25882,2.06,Premium,I,SI2,60.1,58.0,14982,8.32,8.26,4.98
25883,2.01,Ideal,H,SI2,62.5,53.9,14998,8.04,8.07,5.04
26611,2.05,Premium,G,SI2,60.1,59.0,16357,8.2,8.3,4.96
26458,2.02,Premium,H,SI2,59.9,55.0,15996,8.28,8.17,4.93
20983,1.71,Premium,H,SI1,58.1,59.0,9193,7.88,7.81,4.56
22389,2.02,Ideal,I,SI2,62.2,57.0,10412,8.06,7.99,4.99
27090,2.15,Premium,H,SI2,62.8,58.0,17221,8.22,8.17,5.15
26063,1.77,Premium,E,VS2,61.6,58.0,15278,7.78,7.71,4.77
26617,2.28,Premium,J,VS2,62.4,58.0,16369,8.45,8.35,5.24
21815,1.75,Ideal,J,VS2,62.1,56.0,9890,7.74,7.69,4.79
24887,2.06,Premium,G,SI1,59.3,61.0,13317,8.44,8.36,4.98
26079,2.04,Ideal,I,SI1,60.0,60.0,15308,8.3,8.26,4.97
24966,2.02,Premium,H,SI1,63.0,60.0,13453,7.85,7.79,4.93
"""

# Read the data using pandas
diamonds = pd.read_csv(io.StringIO(data_str))

# Create "bump" dummies
diamonds["carat_05"] = (diamonds["carat"] >= 0.5).astype(int)
diamonds["carat_10"] = (diamonds["carat"] >= 1.0).astype(int)
diamonds["carat_15"] = (diamonds["carat"] >= 1.5).astype(int)
diamonds["carat_20"] = (diamonds["carat"] >= 2.0).astype(int)

app_ui = ui.page_fluid(
    ui.h2("Diamond Price Modeling"),
    ui.layout_sidebar(
        ui.sidebar(
            ui.input_slider("num_points", "Number of Simulated Points", 
                           min=100, max=1000, value=200, step=100),
            ui.input_slider("noise_level", "Base Noise Level", 
                           min=500, max=3000, value=1000, step=100),
            ui.input_checkbox("show_original", "Show Original Data", True),
            ui.input_checkbox("show_simulated", "Show Simulated Data", True),
            width=250
        ),
        ui.output_plot("diamondPlot", height="500px"),
    ),
)

def server(input, output, session):
    
    @reactive.Calc
    def generate_simulated_data():
        # Fit a linear model to the real data
        # X = diamonds['carat'].values.reshape(-1, 1)
        # y = diamonds['price'].values
        # # THIS ISN'T ACTUALLY USING THE NEW MODEL
        # coefficients = np.polyfit(X.flatten(), y, 1)
        # slope, intercept = coefficients

        # Extract data
        X = diamonds['carat'].values  # No need to reshape for statsmodels
        y = diamonds['price'].values

        # Create a DataFrame for statsmodels
        data = pd.DataFrame({'carat': X, 'price': y})

        # Fit a GLM using the Gamma family with an identity link function
        glm_model = smf.glm("price ~ carat", data=data, family=sm.families.Gamma(sm.families.links.identity())).fit()

        # Extract estimated coefficients
        intercept, slope = glm_model.params
        # End revision of model

        # Generate new X values
        X_new = np.linspace(diamonds['carat'].min(), diamonds['carat'].max(), input.num_points())
        
        # Generate noise that scales with X
        base_noise = np.random.normal(0, input.noise_level(), len(X_new))
        scaled_noise = X_new * base_noise  # Multiply noise by carat value
        
        # Generate Y values with scaled noise
        Y_new = intercept + slope * X_new + scaled_noise
        
        return pd.DataFrame({'carat': X_new, 'price': Y_new})

    @output
    @render.plot
    def diamondPlot():
        plt.clf()
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111)
        
        if input.show_original():
            ax.scatter(diamonds['carat'], diamonds['price'], 
                      alpha=0.6, color='blue', label='Original Data')
        
        if input.show_simulated():
            simulated_data = generate_simulated_data()
            ax.scatter(simulated_data['carat'], simulated_data['price'], 
                      alpha=0.6, color='orange', label='Simulated Data')
        
        ax.set_title('Diamond Carat vs Price: Original and Simulated Data\nwith Carat-Dependent Noise')
        ax.set_xlabel('Carat')
        ax.set_ylabel('Price')
        ax.grid(True)
        ax.legend()
        
        fig.canvas.draw_idle()
        plt.close()
        
        return fig

app = App(app_ui, server)
```